# Introduction
- This introduction is the easiest way for Python folks to play with Percolate
- We provider the Postgres client using the Docker compose connection parameteres as defaults
- We can learn how to register entities after which everyone else becomes possible


In [1]:

import sys
sys.path.append('../')
import requests
#requests.get('http://www.percolationlabs.ai').content

In [2]:
from percolate.models.p8 import sample_models, LanguageModelApi
import percolate as p8
from percolate.services import PostgresService
from percolate.models.p8 import PlanModel

p8.repository(PlanModel).register()
#p8.repository(LanguageModelApi).update_records(sample_models)

In [3]:
# pg.execute(f""" select * from percolate_with_agent('how does percolate manage to work with google, openai and anthropic schemes seamlessly in the database - give sql examples',
#  'p8.PercolateAgent') """)

In [ ]:
from percolate.models.p8 import PercolateAgent

agent = p8.Agent(PercolateAgent)

agent("What is percolate")

In [ ]:
agent("What is percolate - please procide a 5 page description of percolate and how it compares to other databaes")

In [ ]:
from IPython.display import Markdown
#Markdown(agent("Are there any functions for natural langauge to sql? - can you give me an example of how to use it"))
Markdown(agent("Are there any functions in Percolate for using claude and other models questions - give me an example of how to use it"))


In [ ]:
data = pg.execute(""" select * from p8.query_entity('what sql queries do we have for generating uuids from json', 'p8.PercolateAgent')  """)
data

In [ ]:
from IPython.display import Markdown
'what sql queries do we have for generating uuids from json', 'p8.PercolateAgent'

df = pg.execute(f"""  select * from p8.query_entity('what sql queries do we have for generating uuids from json', 'p8.PercolateAgent')  """) 
df

In [ ]:
agent.search("do we have any fucntions for generating json in sql")

## Registering entities which are Agents is the key way to use Percolate
- entity models describe agents completely
- Percolate assumes agents are serializable in the "Agent" model which has structured response, sytem prompt and external function refs
- We register Entities as tables that can store data, as Agents that can allow interaction with data
- Agents are added as structured tables but other indexes are added too; embeddings on fields that need them and graph node registry

In [4]:
from percolate.models.p8 import Project, Agent, ModelField, LanguageModelApi, Function, Session, AIResponse, ApiProxy, IndexAudit

#we register the core models - these are added by scripts in install anyway but it illustrates for 'bring your own'
for model in [AIResponse,Session, IndexAudit]:
    repo = pg.repository(model)
    repo.register(register_entities=True)

## Percolate configures Langauge models and assumes we can load tokens from the env
- While not recommended in production, for simplicity you might add tokens locally in your database
- The example below registers unique model names - the name is sometimes of the from provider-model if you want to select an API

In [ ]:
"""these are stored in the database by default and you can add your own
in the python client your API token is used
in the database you can save the token but the extension can also load it from the environment if its maintained on the database cluster
"""
from percolate.models.p8 import sample_models

## Adding APIs
- Percolate registers apis and functions 
- for example a freely available test api is at https://petstore.swagger.io/#/pet/findPetsByStatus
- we add this to percolate below

In [3]:
from percolate.utils.ingestion import add 
import os
# add.add_api('swagger_test', 'https://petstore.swagger.io/v2/swagger.json', verbs='get')

In [ ]:
import percolate as p8
from percolate.models.p8 import Function
"""filter by the proxy, verb etc if needed"""
functions = [Function(**f) for f in p8.repository(Function).select()]
"""functions are callable"""

#### any database loaded function can be called and you can use the metadata to know how to call it(as can an llm)

### the function manager is responsible for finding functions in the database but you can also ref external functions on your agent
under the hood if any agent either asks to load a function or if the function is added as a reference in it's `get_functions` the funciton manager is used to load the function into context
```python
from percolate.services import FunctionManager
f = FunctionManager()
f.add_functions_by_key('get_pet_findByStatus')
```

### Example of how we bootsrap the database by adding the P8 models. 

In [3]:
import sys
sys.path.append('../')
import requests
from percolate.models import bootstrap

"""this generates the script that we use for setup of percolate (WIP)"""
bootstrap(root='../../../../extension/', apply=True)